In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/merged-stats-n-scorecards-trimmed/merged_stats_n_scorecards_trimmed.csv


# Feature Transformation

In [66]:
df = pd.read_csv("/kaggle/input/merged-stats-n-scorecards-trimmed/merged_stats_n_scorecards_trimmed.csv", sep=';')
df.head()

,red_fighter_name,blue_fighter_name,event_date,red_fighter_result,blue_fighter_result,method,bout_type,bonus,red_fighter_total_pts,blue_fighter_total_pts
0,ILIA TOPURIA,MAX HOLLOWAY,26/10/2024,W,L,KO/TKO,UFC Featherweight Title Bout,belt,20 20 19,18 18 19
1,ROBERT WHITTAKER,KHAMZAT CHIMAEV,26/10/2024,L,W,Submission,Middleweight Bout,perf,- - -,- - -
2,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,26/10/2024,W,L,Decision - Unanimous,Light Heavyweight Bout,-,29 29 29,28 28 28
3,LERONE MURPHY,DAN IGE,26/10/2024,W,L,Decision - Unanimous,Featherweight Bout,-,29 29 29,28 28 28
4,SHARA MAGOMEDOV,ARMEN PETROSYAN,26/10/2024,W,L,KO/TKO,Middleweight Bout,perf,10 10 10,9 9 9


In [67]:
df['winner'] = df['red_fighter_result'].apply(lambda x: "red" if x == 'W' else "blue")
df.drop(['event_date', 'bonus', 'red_fighter_result', 'blue_fighter_result'], axis=1, inplace=True)

# Changing columns orders
df = df.loc[:, ['red_fighter_name', 'blue_fighter_name', 'winner', 'method', 'bout_type', 'red_fighter_total_pts', 'blue_fighter_total_pts']]
df.head()

,red_fighter_name,blue_fighter_name,winner,method,bout_type,red_fighter_total_pts,blue_fighter_total_pts
0,ILIA TOPURIA,MAX HOLLOWAY,red,KO/TKO,UFC Featherweight Title Bout,20 20 19,18 18 19
1,ROBERT WHITTAKER,KHAMZAT CHIMAEV,blue,Submission,Middleweight Bout,- - -,- - -
2,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,red,Decision - Unanimous,Light Heavyweight Bout,29 29 29,28 28 28
3,LERONE MURPHY,DAN IGE,red,Decision - Unanimous,Featherweight Bout,29 29 29,28 28 28
4,SHARA MAGOMEDOV,ARMEN PETROSYAN,red,KO/TKO,Middleweight Bout,10 10 10,9 9 9


## Extracting only lightweight division

In [68]:
lightweight_division_filter = df.apply(lambda x: 'Lightweight' in x['bout_type'], axis=1)
df_lightweight_division = df[lightweight_division_filter]
df_lightweight_division.reset_index(drop=True, inplace=True)
df_lightweight_division.head()

,red_fighter_name,blue_fighter_name,winner,method,bout_type,red_fighter_total_pts,blue_fighter_total_pts
0,GRANT DAWSON,RAFA GARCIA,red,KO/TKO,Lightweight Bout,10 10 10,9 9 9
1,AUSTIN HUBBARD,ALEXANDER HERNANDEZ,blue,Decision - Split,Lightweight Bout,30 28 28,27 29 29
2,RENATO MOICANO,BENOIT SAINT DENIS,red,TKO - Doctor's Stoppage,Lightweight Bout,19 19 19,18 18 18
3,FARES ZIAM,MATT FREVOLA,red,KO/TKO,Lightweight Bout,20 20 20,18 18 17
4,LUDOVIT KLEIN,ROOSEVELT ROBERTS,red,Decision - Unanimous,Lightweight Bout,30 29 29,27 28 28


## Summing the points up

In [69]:
import re

def sum_points(row):
    if re.match(r"\d", row):
        return sum(int(num) for num in row.split())
    else:
        return "-"

df_lightweight_division.loc[:, 'red_fighter_total_pts'] = df_lightweight_division['red_fighter_total_pts'].apply(sum_points)
df_lightweight_division.loc[:, 'blue_fighter_total_pts'] = df_lightweight_division['blue_fighter_total_pts'].apply(sum_points)

df_lightweight_division.head()

,red_fighter_name,blue_fighter_name,winner,method,bout_type,red_fighter_total_pts,blue_fighter_total_pts
0,GRANT DAWSON,RAFA GARCIA,red,KO/TKO,Lightweight Bout,30,27
1,AUSTIN HUBBARD,ALEXANDER HERNANDEZ,blue,Decision - Split,Lightweight Bout,86,85
2,RENATO MOICANO,BENOIT SAINT DENIS,red,TKO - Doctor's Stoppage,Lightweight Bout,57,54
3,FARES ZIAM,MATT FREVOLA,red,KO/TKO,Lightweight Bout,60,53
4,LUDOVIT KLEIN,ROOSEVELT ROBERTS,red,Decision - Unanimous,Lightweight Bout,88,83


## Winner and Loser instead of Red / Blue fighter

In [70]:
df_lightweight_division = df_lightweight_division.copy()
df_lightweight_division.loc[:, 'winner_name'] = df_lightweight_division.apply(lambda row: row['red_fighter_name'] if row['winner'] == 'red' else row['blue_fighter_name'], axis=1)
df_lightweight_division.loc[:, 'loser_name'] = df_lightweight_division.apply(lambda row: row['red_fighter_name'] if row['winner'] == 'blue' else row['blue_fighter_name'], axis=1)
df_lightweight_division.loc[:, 'winner_total_pts'] = df_lightweight_division.apply(lambda row: row['red_fighter_total_pts'] if row['winner'] == 'red' else row['blue_fighter_total_pts'], axis=1)
df_lightweight_division.loc[:, 'loser_total_pts'] = df_lightweight_division.apply(lambda row: row['red_fighter_total_pts'] if row['winner'] == 'blue' else row['blue_fighter_total_pts'], axis=1)

df_lightweight_division = df_lightweight_division.loc[:, ['winner_name', 'loser_name', 'method', 'bout_type', 'winner_total_pts', 'loser_total_pts']]
df_lightweight_division.head()

,winner_name,loser_name,method,bout_type,winner_total_pts,loser_total_pts
0,GRANT DAWSON,RAFA GARCIA,KO/TKO,Lightweight Bout,30,27
1,ALEXANDER HERNANDEZ,AUSTIN HUBBARD,Decision - Split,Lightweight Bout,85,86
2,RENATO MOICANO,BENOIT SAINT DENIS,TKO - Doctor's Stoppage,Lightweight Bout,57,54
3,FARES ZIAM,MATT FREVOLA,KO/TKO,Lightweight Bout,60,53
4,LUDOVIT KLEIN,ROOSEVELT ROBERTS,Decision - Unanimous,Lightweight Bout,88,83


# ELO Algorithm

## Creating the leaderboard

In [71]:
all_fighters = set(pd.concat([df_lightweight_division['winner_name'], df_lightweight_division['loser_name']]))
print(f"Total number of unique fighters in the lightweight division: {len(all_fighters)}")

Total number of unique fighters in the lightweight division: 164


## 1500 is the base ELO score for all fighters

In [72]:
lightweight_leaderboard = {fighter: 1500 for fighter in all_fighters}
dict(list(lightweight_leaderboard.items())[:5])

{'DAN HOOKER': 1500,
 'MATT FREVOLA': 1500,
 'MANUEL TORRES': 1500,
 'ROLANDO BEDOYA': 1500,
 'RAFAEL FIZIEV': 1500}

## Defining all parts of the algorithm

In [73]:
def expected_probability(elo_fighter_A, elo_fighter_B):
    prob_fighter_A = 1 / (1 + 10 ** ((elo_fighter_B - elo_fighter_A) / 400))
    prob_fighter_B = 1 - prob_fighter_A
    return prob_fighter_A, prob_fighter_B

def base_K_value(fight_type):
    K_base = 5
    title_fight_multiplier = 1.5
    
    if fight_type == 'UFC Lightweight Title Bout':
        return K_base * title_fight_multiplier
    else:
        return K_base

def victory_method(outcome):
    outcomes = {'KO/TKO': 40,
                'Submission': 40,
                "TKO - Doctor's Stoppage": 35,
                'Decision - Unanimous': 30,
                'Decision - Majority': 25,
                'Decision - Split': 20,
                "Draw": 10,
                "No Contest": 5 
               }
    return outcomes[outcome]

def scorecard_contribution(winner_total_pts, loser_total_pts):
    if isinstance(winner_total_pts, int) and isinstance(loser_total_pts, int):
        winner_R_score = winner_total_pts / (winner_total_pts + loser_total_pts)
        loser_R_score = loser_total_pts / (loser_total_pts + winner_total_pts)
    else:
        winner_R_score = 1
        loser_R_score = 1
        
    return winner_R_score, loser_R_score

def K_adjusted(row):
    # Winner & Loser pre-fight ELO
    winner_ELO = lightweight_leaderboard[row['winner_name']]
    loser_ELO = lightweight_leaderboard[row['loser_name']]
    
    # Calculating K_base
    K_base = base_K_value(row['bout_type'])
    
    # Calculating M
    M = victory_method(row['method'])

    # Calculating scorecard contribution
    R_score_winner, R_score_loser = scorecard_contribution(row['winner_total_pts'], row['loser_total_pts'])
    
    # Final K_adjuste
    K_adjusted_winner = K_base * M * R_score_winner
    K_adjusted_loser = K_base * M * R_score_loser

    # Calculating expected probability
    winner_expected_probability, loser_expected_probability = expected_probability(winner_ELO, loser_ELO)

    # Finally calculating the new ELO
    winner_new_elo = winner_ELO + K_adjusted_winner * (1 - winner_expected_probability)
    loser_new_elo = loser_ELO - K_adjusted_loser * loser_expected_probability
    return winner_new_elo, loser_new_elo

## Stitching it up together

In [74]:
testing_sample = df_lightweight_division.iloc[1]

## ELO Update

In [75]:
def update_ELO(row):
    # Receiving the new ELO
    winner_new_elo, loser_new_elo = K_adjusted(row)

    # Updating the ELO
    lightweight_leaderboard[row['winner_name']] = winner_new_elo
    lightweight_leaderboard[row['loser_name']] = loser_new_elo

In [76]:
df_lightweight_division.apply(update_ELO, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
272    None
273    None
274    None
275    None
276    None
Length: 277, dtype: object

In [86]:
lightweight_leaderboard = pd.Series(lightweight_leaderboard)
lightweight_leaderboard.sort_values(ascending=False)[:20]

ISLAM MAKHACHEV        1846.970353
PADDY PIMBLETT         1782.376684
MANUEL TORRES          1728.861392
JOEL ALVAREZ           1726.520598
LUDOVIT KLEIN          1724.577246
ARMAN TSARUKYAN        1724.521896
BENOIT SAINT DENIS     1718.525853
RAFAEL FIZIEV          1713.996205
TERRANCE MCKINNEY      1708.519701
CHARLES OLIVEIRA       1705.977117
JALIN TURNER           1696.399320
DRAKKAR KLOSE          1680.577120
MATEUSZ GAMROT         1678.088120
CHASE HOOPER           1666.887818
ALEXANDER HERNANDEZ    1642.686166
IGNACIO BAHAMONDES     1637.331246
GABRIEL BENITEZ        1633.031470
MATEUSZ REBECKI        1627.471833
CLAUDIO PUELLES        1626.843995
JORDAN LEAVITT         1625.517020
dtype: float64

In [90]:
lightweight_leaderboard['GRANT DAWSON']

1605.5657693417002